In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

Mounted at /content/drive


In [ ]:
# Initialize stroke folder paths
stroke_folders = ["BK1", "BR1", "FL1", "FR1", "BK2", "BR2", "FL2", "FR2"]
folder_path = "/content/drive/MyDrive/Colab Notebooks/CPSC 490/Images/"

# Initialize array for image paths and data array
image_paths = []
data_array = []
big_df = pd.DataFrame()
# loop through folders
for folder in stroke_folders:
  os.chdir(folder_path + folder)
  segment_folders = [x for x in os.listdir() if "Segment" in x]

  stroke_data_array = []
  # loop through segments to read all excel files
  for segment in segment_folders:
    labels_file = folder_path + folder + "/" + segment + "/labels.xlsx"
    # read excel file and append image path of frame to image pathslist
    df = pd.read_excel(labels_file, header=None)
    image_paths.extend([folder_path + folder + "/" + segment + "/"] * 30)
    # add spreadsheet labels to big dataframe
    big_df = pd.concat([big_df, df], ignore_index=True)


In [ ]:
# define column names
column_names = ['image', 'stroke', 'completion', 'vid_segment']
big_df.columns = column_names

# loop through columns of big dataframe and add to data array
for col in column_names:
    data_array.append(big_df[col])

# use numpy to convert data array to np array
data_array = np.array(data_array)

In [ ]:
image_paths = [str(x) + str(y) for x, y in zip(image_paths, data_array[0])]

In [ ]:
class StrokeTypeTrainingDataset(torch.utils.data.Dataset):
    '''
    Dataset for fetching:
        (1) image
        (2) label (integer 0-3)

    Arg(s):
        image_paths : list[str]
            paths to image
        labels : list[int]
            labels containing stroke types
            0: butterfly
            1: backstroke
            2: breaststroke
            3: freestyle
        resize_shape : tuple[int]
            shape (height, width) to resize inputs
    '''

    def __init__(self,
                 image_paths,
                 labels,
                 resize_shape):

        self.image_paths = image_paths
        self.labels = labels
        self.resize_shape = resize_shape

    def __getitem__(self, index):
        # Load image
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert("RGB")

        # Convert from PIL image to numpy array
        image = np.array(image)

        # Load label
        label = np.array(self.labels[index])

        # Resize image to resize_shape
        image = cv2.resize(
              image,
              [self.resize_shape[1], self.resize_shape[0]],
              interpolation = cv2.INTER_LINEAR)

        image = np.transpose(image, [2, 0, 1])

        # Convert to float32
        image = image.astype(np.float32)
        label = label.astype(np.int64)

        return image, label

    def __len__(self):
        return len(self.image_paths)

In [ ]:
class StrokeCompletionTrainingDataset(torch.utils.data.Dataset):
    '''
    Dataset for fetching:
        (1) image
        (2) label (integer 0-1)

    Arg(s):
        image_paths : list[str]
            paths to image
        labels : list[int]
            labels containing stroke completion
            0: stroke not complete
            1: stroke complete
        resize_shape : tuple[int]
            shape (height, width) to resize inputs
    '''

    def __init__(self,
                 image_paths,
                 labels,
                 resize_shape):

        self.image_paths = image_paths
        self.labels = labels
        self.resize_shape = resize_shape

    def __getitem__(self, index):
        # Load image
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert("RGB")

        # Convert from PIL image to numpy array
        image = np.array(image)

        # Load label
        label = np.array(self.labels[index])

        # Resize image to resize_shape
        image = cv2.resize(
              image,
              [self.resize_shape[1], self.resize_shape[0]],
              interpolation = cv2.INTER_LINEAR)

        image = np.transpose(image, [2, 0, 1])

        # Convert to float32
        image = image.astype(np.float32)
        label = label.astype(np.int64)

        return image, label

    def __len__(self):
        return len(self.image_paths)

NameError: name 'torch' is not defined

In [ ]:
class VideoSegmentTrainingDataset(torch.utils.data.Dataset):
    '''
    Dataset for fetching:
        (1) image
        (2) label (integer 0-4)

    Arg(s):
        image_paths : list[str]
            paths to image
        labels : list[int]
            labels containing video segment
            0: not swimming
            1: underwater
            2: swimming
            3: turn
        resize_shape : tuple[int]
            shape (height, width) to resize inputs
    '''

    def __init__(self,
                 image_paths,
                 labels,
                 resize_shape):

        self.image_paths = image_paths
        self.labels = labels
        self.resize_shape = resize_shape

    def __getitem__(self, index):
        # Load image
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert("RGB")

        # Convert from PIL image to numpy array
        image = np.array(image)

        # Load label
        label = np.array(self.labels[index])

        # Resize image to resize_shape
        image = cv2.resize(
              image,
              [self.resize_shape[1], self.resize_shape[0]],
              interpolation = cv2.INTER_LINEAR)

        image = np.transpose(image, [2, 0, 1])

        # Convert to float32
        image = image.astype(np.float32)
        label = label.astype(np.int64)

        return image, label

    def __len__(self):
        return len(self.image_paths)

In [ ]:
# Define training split
train_split_size = int(0.80 * len(image_paths))

# Define image paths and labels
image_paths = np.array(image_paths)
labels_stroke = np.array(data_array[1])
labels_completion = np.array(data_array[2])
labels_vid_seg = np.array(data_array[3])

# Divide images into training and testing images
train_image_paths = image_paths[0: train_split_size]
test_image_paths = image_paths[train_split_size:]

# Divide labels into training and testing labels
train_labels_stroke = labels_stroke[0: train_split_size]
train_labels_completion = labels_completion[0: train_split_size]
# train_labels_vid_seg = labels_vid_seg[0: train_split_size]

test_labels_stroke = labels_stroke[train_split_size:]
test_labels_completion = labels_completion[train_split_size:]
# test_labels_vid_seg = labels_vid_seg[train_split_size:]

# Define datasets for each set of training/testing images and labels
training_stroke_dataset = StrokeTypeTrainingDataset(image_paths=train_image_paths, labels=train_labels_stroke, resize_shape=(256,256))
training_completion_dataset = StrokeCompletionTrainingDataset(image_paths=train_image_paths, labels=train_labels_completion, resize_shape=(256,256))
# training_vid_seg_dataset = VideoSegmentTrainingDataset(image_paths=train_image_paths, labels=train_labels_vid_seg, resize_shape=(256,256))

testing_stroke_dataset = StrokeTypeTrainingDataset(image_paths=test_image_paths, labels=test_labels_stroke, resize_shape=(256,256))
testing_completion_dataset = StrokeTypeTrainingDataset(image_paths=test_image_paths, labels=test_labels_completion, resize_shape=(256,256))

# Define batch size
batch_size = 25

# Define dataloaders for each training and testing dataset
training_stroke_dataloader = DataLoader(training_stroke_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
testing_stroke_dataloader = DataLoader(testing_stroke_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=1)

training_completion_dataloader = DataLoader(training_completion_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
testing_completion_dataloader = DataLoader(testing_completion_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=1)